In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline 
%config IPCompleter.greedy=True

import collections
import io
import math
import tensorflow as tf
from IPython import display
from sklearn import metrics


## Reading data from file

In [2]:
from collections import Counter

### main dataset ###
data = pd.read_csv("20151219.txt", sep = "\t", header = None)
data.columns = ["conn_len", "service", "src_bytes", "dst_bytes", "conn_numb", "Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "Conn_state", "IDS_detection", "Malware_detection", "Ashula_detection", "attack_flag", "src_IP", "src_port", "dst_IP", "dst_port", "start_time", "proto"]

print(Counter(data["attack_flag"]))

Counter({-1: 149915, 1: 67713})


## Transform to binary

In [3]:
data.loc[:, "attack_flag"] = data.loc[:,"attack_flag"].replace(-2, -1)
data.loc[:, "attack_flag"] = data.loc[:, "attack_flag"] * -1
data.loc[:, "attack_flag"] = data.loc[:,"attack_flag"].replace(-1, 0)

print(Counter(data["attack_flag"]))

Counter({1: 149915, 0: 67713})


In [4]:
# removing NaNs
data = data.dropna()
data.shape

(217628, 24)

# a bit of feature engineering

## port numbers classification

In [5]:
t1 = 1023  # well-known port numbers
t2 = 49151 # registered ports
t3 = 65535 # client ports

def wk(data_row):
    if (data_row["src_port"] <= t1): 
        value = 1
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 0
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 0
    return value

def reg(data_row):
    if (data_row["src_port"] <= t1): 
        value = 0
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 1
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 0
    return value

def cli(data_row):
    if (data_row["src_port"] <= t1): 
        value = 0
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 0
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 1
    return value

data["well_known_src_pool"] = data.apply(wk, axis=1)
data["registered_src_pool"] = data.apply(reg, axis=1)
data["cli_src_pool"] = data.apply(cli, axis=1)

In [6]:
data["src_port"] = data["src_port"].apply(str)
data["dst_port"] = data["dst_port"].apply(str)

In [7]:
train, validate, test = np.split(data, [int(.7*len(data)), int(.85*len(data))])

train = train.sample(frac=1)
validate = validate.sample(frac=1)
test = test.sample(frac=1)

y_train = train.loc[:, "attack_flag"].values.ravel()
X_train = train.drop(["attack_flag"], axis=1)
y_validate = validate.loc[:, "attack_flag"].values.ravel()
X_validate = validate.drop(["attack_flag"], axis=1)
y_test = test.loc[:, "attack_flag"].values.ravel()
X_test = test.drop(["attack_flag"], axis=1)
print(train.shape)
print(validate.shape)
print(test.shape)

(152339, 27)
(32644, 27)
(32645, 27)


In [8]:
deep_numeric_feat = ["Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "well_known_src_pool", "registered_src_pool", "cli_src_pool"]
wide_numeric_feat = ["conn_len",  "src_bytes", "dst_bytes", "conn_numb"]
categorical_feat = ["service", "Conn_state", "src_port", "dst_port", "proto"]

In [9]:
deep_numeric_feat_tf = [tf.feature_column.numeric_column(k) for k in deep_numeric_feat]
wide_numeric_feat_tf = [tf.feature_column.numeric_column(k) for k in wide_numeric_feat]

In [10]:
#data["src_port"].unique().tolist()

In [146]:
src_port_mod = ["src_" + s for s in data["src_port"].unique().tolist()]
src_port = tf.feature_column.categorical_column_with_vocabulary_list(
    'src_port', src_port_mod)

dst_port_mod = ["dst_" + s for s in data["dst_port"].unique().tolist()]
dst_port = tf.feature_column.categorical_column_with_vocabulary_list(
    'dst_port', dst_port_mod)

service = tf.feature_column.categorical_column_with_vocabulary_list(
    'service', data["service"].unique().tolist())

Conn_state = tf.feature_column.categorical_column_with_vocabulary_list(
    'Conn_state', data["Conn_state"].unique().tolist())

proto = tf.feature_column.categorical_column_with_vocabulary_list(
    'proto', data["proto"].unique().tolist())

In [253]:
deep_columns = [
    tf.feature_column.indicator_column(service),
    tf.feature_column.indicator_column(Conn_state),
    tf.feature_column.indicator_column(proto),
    tf.feature_column.embedding_column(src_port, dimension=10),
    tf.feature_column.embedding_column(dst_port, dimension=6),
]

In [265]:
crossed_columns = [
      tf.feature_column.crossed_column(
          ['src_port', 'service'], hash_bucket_size=50000),
      tf.feature_column.crossed_column(
          ['dst_port', 'service'], hash_bucket_size=50000),
      tf.feature_column.crossed_column(
          ['conn_len', 'src_bytes'], hash_bucket_size=50000),
      tf.feature_column.crossed_column(
          ['conn_len', 'dst_bytes'], hash_bucket_size=50000),
      tf.feature_column.crossed_column(
          ['dst_port', 'conn_numb'], hash_bucket_size=50000),  
      tf.feature_column.crossed_column(
          ['service', 'proto'], hash_bucket_size=50),
  ]

In [278]:
model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir = "/home/matz/Desktop/ml_ids/NN_model2",
    dnn_activation_fn=tf.nn.selu,
    linear_feature_columns = wide_numeric_feat_tf + crossed_columns + deep_columns,
    dnn_feature_columns = deep_columns + deep_numeric_feat_tf, # + wide_numeric_feat_tf,
    dnn_hidden_units = [2048, 256]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/matz/Desktop/ml_ids/NN_model2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc9eb3fed30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [279]:
feature_set= deep_numeric_feat + wide_numeric_feat + categorical_feat

In [280]:
def get_input_fn(data_set, num_epochs=None, n_batch = 256, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: data_set[k].values for k in feature_set}),
       y = pd.Series(data_set["attack_flag"].values),
       batch_size=n_batch,   
       num_epochs=num_epochs,
       shuffle=shuffle)

In [281]:
model.train(input_fn=get_input_fn(train, n_batch=180), steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /home/matz/Desktop/ml_ids/NN_model2/model.ckpt.
INFO:tensorflow:loss = 90.93876, step = 1
INFO:tensorflow:global_step/sec: 47.6309
INFO:tensorflow:loss = 3.2304542, step = 101 (2.101 sec)
INFO:tensorflow:global_step/sec: 62.1071
INFO:tensorflow:loss = 3.563142, step = 201 (1.610 sec)
INFO:tensorflow:global_step/sec: 60.3127
INFO:tensorflow:loss = 13.081627, step = 301 (1.658 sec)
INFO:tensorflow:global_step/sec: 64.9024
INFO:tensorflow:loss = 4.7100477, step = 401 (1.540 sec)
INFO:tensorflow:global_step/sec: 65.7336
INFO:tensorflow:loss = 4.142282, step = 501 (1.521 sec)
INFO:tensorflow:global_step/sec: 76.7566
INFO:tensorflow:loss = 4.0614605, step = 601 (1.303 sec)
INFO:tensorflow:global_step/sec: 82.6864


INFO:tensorflow:global_step/sec: 79.034
INFO:tensorflow:loss = 16.112827, step = 8101 (1.265 sec)
INFO:tensorflow:global_step/sec: 80.7343
INFO:tensorflow:loss = 9.67308, step = 8201 (1.239 sec)
INFO:tensorflow:global_step/sec: 79.7729
INFO:tensorflow:loss = 7.6600485, step = 8301 (1.253 sec)
INFO:tensorflow:global_step/sec: 80.2627
INFO:tensorflow:loss = 1.9371315, step = 8401 (1.246 sec)
INFO:tensorflow:global_step/sec: 80.7393
INFO:tensorflow:loss = 3.3619463, step = 8501 (1.239 sec)
INFO:tensorflow:global_step/sec: 76.7333
INFO:tensorflow:loss = 2.8607569, step = 8601 (1.303 sec)
INFO:tensorflow:global_step/sec: 79.2325
INFO:tensorflow:loss = 9.186527, step = 8701 (1.262 sec)
INFO:tensorflow:global_step/sec: 76.3439
INFO:tensorflow:loss = 7.030637, step = 8801 (1.310 sec)
INFO:tensorflow:global_step/sec: 80.7679
INFO:tensorflow:loss = 2.495896, step = 8901 (1.238 sec)
INFO:tensorflow:global_step/sec: 79.9478
INFO:tensorflow:loss = 2.1449394, step = 9001 (1.251 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 78.2982
INFO:tensorflow:loss = 8.739373, step = 16401 (1.277 sec)
INFO:tensorflow:global_step/sec: 79.4607
INFO:tensorflow:loss = 5.423849, step = 16501 (1.259 sec)
INFO:tensorflow:global_step/sec: 80.8838
INFO:tensorflow:loss = 1.4996603, step = 16601 (1.237 sec)
INFO:tensorflow:global_step/sec: 79.82
INFO:tensorflow:loss = 1.7778836, step = 16701 (1.253 sec)
INFO:tensorflow:global_step/sec: 80.3256
INFO:tensorflow:loss = 2.1943467, step = 16801 (1.245 sec)
INFO:tensorflow:global_step/sec: 81.4729
INFO:tensorflow:loss = 2.8203535, step = 16901 (1.228 sec)
INFO:tensorflow:global_step/sec: 78.8195
INFO:tensorflow:loss = 6.5819025, step = 17001 (1.269 sec)
INFO:tensorflow:global_step/sec: 78.6694
INFO:tensorflow:loss = 0.65388215, step = 17101 (1.271 sec)
INFO:tensorflow:global_step/sec: 79.8094
INFO:tensorflow:loss = 2.172902, step = 17201 (1.253 sec)
INFO:tensorflow:global_step/sec: 79.6016
INFO:tensorflow:loss = 22.88455, step = 17301 (1.256 sec)
INFO:

In [282]:
eval_metrics = model.evaluate(input_fn=get_input_fn(validate, n_batch=validate.shape[0], shuffle=False), steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-17-09:54:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/matz/Desktop/ml_ids/NN_model2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2019-05-17-09:54:17
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9975493, accuracy_baseline = 0.7026406, auc = 0.9995263, auc_precision_recall = 0.9986777, average_loss = 0.011711188, global_step = 20000, label/mean = 0.2973594, loss = 382.30002, precision = 0.99424994, prediction/mean = 0.29791233, recall = 0.99752754
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /home/matz/Desktop/ml_ids/NN_model2/model.ckpt-20000


In [283]:
for key,value in sorted(eval_metrics.items()):
  print('%s: %s' % (key, value))

accuracy: 0.9975493
accuracy_baseline: 0.7026406
auc: 0.9995263
auc_precision_recall: 0.9986777
average_loss: 0.011711188
global_step: 20000
label/mean: 0.2973594
loss: 382.30002
precision: 0.99424994
prediction/mean: 0.29791233
recall: 0.99752754


In [284]:
results = []
results = list(model.predict(input_fn=get_input_fn(test, num_epochs=1, n_batch = X_test.shape[0], shuffle=False)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/matz/Desktop/ml_ids/NN_model2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [285]:
y_pred = []
for el in results:
    y_pred.append(el["class_ids"])

In [286]:
name = "NN"

Acc = {}
F1S = {}
Prec = {}
Rec = {}
FPR = {}

Acc[name] = metrics.accuracy_score(test["attack_flag"], y_pred)
F1S[name] = metrics.f1_score(test["attack_flag"], y_pred)
Prec[name] = metrics.precision_score(test["attack_flag"], y_pred)
Rec[name] = metrics.recall_score(test["attack_flag"], y_pred)
tn, fp, fn, tp = metrics.confusion_matrix(test["attack_flag"], y_pred).ravel()
FPR[name] = fp/(fp+tn)

print("{0:2} Accuracy: {1:.5f}, F1-score: {2:.5f}, Precision: {3:.5f}, Recall: {4:.5f}, FPR: {5:.5f}".format(name, Acc[name], F1S[name], Prec[name], Rec[name], FPR[name]))
print("TN: {0:3}; FP: {1:3}; FN: {2:4}; TP: {3:3}\n".format(tn, fp, fn, tp))

NN Accuracy: 0.98441, F1-score: 0.99177, Precision: 0.99221, Recall: 0.99134, FPR: 0.14185
TN: 1458; FP: 241; FN:  268; TP: 30678



## New data prediction

In [185]:
test_set = pd.read_csv("20151220.txt", sep = "\t", header = None)
test_set.columns = ["conn_len", "service", "src_bytes", "dst_bytes", "conn_numb", "Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "Conn_state", "IDS_detection", "Malware_detection", "Ashula_detection", "attack_flag", "src_IP", "src_port", "dst_IP", "dst_port", "start_time", "proto"]
print(Counter(test_set["attack_flag"]))

Counter({-1: 189082, 1: 12436})


In [113]:
test_set.loc[:, "attack_flag"] = test_set.loc[:,"attack_flag"].replace(-2, -1)
test_set.loc[:, "attack_flag"] = test_set.loc[:, "attack_flag"] * -1
test_set.loc[:, "attack_flag"] = test_set.loc[:,"attack_flag"].replace(-1, 0)

In [114]:
# removing NaNs
test_set = test_set.dropna()
test_set.shape

(201518, 24)

In [115]:
test_set["well_known_src_pool"] = test_set.apply(wk, axis=1)
test_set["registered_src_pool"] = test_set.apply(reg, axis=1)
test_set["cli_src_pool"] = test_set.apply(cli, axis=1)

In [116]:
test_set["src_port"] = test_set["src_port"].apply(str)
test_set["dst_port"] = test_set["dst_port"].apply(str)

In [117]:
#test_new_data = test_set.sample(n=30000, random_state=1)
test_new_data = test_set.loc[range(30000), :]
test_new_data = test_new_data.sample(frac=1)
y_test_new_data = test_new_data.loc[:, "attack_flag"].values.ravel()
X_test_new_data = test_new_data.drop(["attack_flag"], axis=1)
print(test_new_data.shape)

(30000, 27)


In [118]:
print(Counter(test_new_data["attack_flag"]))

Counter({1: 29540, 0: 460})


In [119]:
results = []
results = list(model.predict(input_fn=get_input_fn(test_new_data, num_epochs=1, n_batch = X_test.shape[0], shuffle=False)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/matz/Desktop/ml_ids/NN_model2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [33]:
y_pred = []
for el in results:
    y_pred.append(el["class_ids"])

In [34]:
name = "NN"

Acc = {}
F1S = {}
Prec = {}
Rec = {}
FPR = {}

Acc[name] = metrics.accuracy_score(y_test_new_data, y_pred)
F1S[name] = metrics.f1_score(y_test_new_data, y_pred)
Prec[name] = metrics.precision_score(y_test_new_data, y_pred)
Rec[name] = metrics.recall_score(y_test_new_data, y_pred)
tn, fp, fn, tp = metrics.confusion_matrix(y_test_new_data, y_pred).ravel()
FPR[name] = fp/(fp+tn)

print("{0:2} Accuracy: {1:.5f}, F1-score: {2:.5f}, Precision: {3:.5f}, Recall: {4:.5f}, FPR: {5:.5f}".format(name, Acc[name], F1S[name], Prec[name], Rec[name], FPR[name]))
print("TN: {0:3}; FP: {1:3}; FN: {2:4}; TP: {3:3}\n".format(tn, fp, fn, tp))

NN Accuracy: 0.98410, F1-score: 0.99192, Precision: 0.99284, Recall: 0.99100, FPR: 0.45870
TN: 249; FP: 211; FN:  266; TP: 29274

